In [1]:
# importing the necessary packages
import pandas as pd
import numpy as np 
import requests
from random import randint
from time import sleep
from bs4 import BeautifulSoup

In [2]:
district = {'Quận 1':'quan-1','Quận 2':'quan-2','Quận 3':'quan-3','Quận 4':'quan-4','Quận 5':'quan-5','Quận 6':'quan-6','Quận 7':'quan-7','Quận 8':'quan-8','Quận 9':'quan-9','Quận 10':'quan-10','Quận 11':'quan-11','Quận 12':'quan-12','Quận Bình Tân':'quan-binh-tan','Quận Bình Thạnh':'quan-binh-thanh','Quận Gò Vấp':'quan-go-vap','Quận Phú Nhuận':'quan-phu-nhuan','Quận Tân Bình':'quan-tan-binh','Quận Tân Phú':'quan-tan-phu','Quận Thủ Đức':'quan-thu-duc','Huyện Củ Chi':'huyen-cu-chi','Huyện Hóc Môn':'huyen-hoc-mon','Huyện Bình Chánh':'huyen-binh-chanh','Huyện Nhà Bè':'huyen-nha-be','Huyện Cần Giờ':'huyen-can-gio'}
page = [10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,0,10,10,10,0]
total_page = {}
n_page = 0
for i in district:
    total_page[district[i]] = page[n_page]
    n_page += 1
total_page

{'quan-1': 10,
 'quan-2': 10,
 'quan-3': 10,
 'quan-4': 10,
 'quan-5': 10,
 'quan-6': 10,
 'quan-7': 10,
 'quan-8': 10,
 'quan-9': 10,
 'quan-10': 10,
 'quan-11': 10,
 'quan-12': 10,
 'quan-binh-tan': 10,
 'quan-binh-thanh': 10,
 'quan-go-vap': 10,
 'quan-phu-nhuan': 10,
 'quan-tan-binh': 10,
 'quan-tan-phu': 10,
 'quan-thu-duc': 10,
 'huyen-cu-chi': 0,
 'huyen-hoc-mon': 10,
 'huyen-binh-chanh': 10,
 'huyen-nha-be': 10,
 'huyen-can-gio': 0}

In [3]:
def get_id_house(district,total_page=10,scale=0):
    """
    Example input: want to get id house at Quan 1 with 50 page
    get_id_house('quan-1',50)
    scale: 0,10,20,...
    """
    if int(total_page) == 0:
        return list()
    id = []
    for page in range(1+scale,int(total_page)+1+scale):
        url = 'https://propzy.vn/mua/bat-dong-san/hcm/'+ str(district) + '/p' + str(page)
        id_r = requests.get(url)
        id_coverpage = id_r.content
        id_soup = BeautifulSoup(id_coverpage, 'html5lib')
        id_class = id_soup.find_all('div', class_='item-listing listing-card view-as-list item-compare')
        for i in id_class:
            id.append(int(i['data-id']))
        print('Got id page',page)
    return id 

In [4]:
def get_attribute_and_price_house(district,total_page=10,scale=0):
    """
    Example input: want to get attribute house at Quan 1 with list_id from get_id_house
    get_attribute_house('quan-1')
    scale: 0,10,20,...
    """
    full_attribute_list = []
    list_id = get_id_house(district,total_page,scale)
    cumsum = 0
    if len(list_id) == 0:
        return list()
    for id in list_id:
        url = 'https://propzy.vn/mua/nha/hcm/' + str(district) + '/id' + str(id)
        attribute_r = requests.get(url)
        attribute_coverpage = attribute_r.content
        attribute_soup = BeautifulSoup(attribute_coverpage)
        attribute = {}
        #Get name_district
        attribute['Địa điểm'] = district
        try:
            #Category type of house
            attribute['Phân Loại'] = attribute_soup.find_all('h1',class_='h3-title')[0].text
            #Get main_attribute
            attribute_class = attribute_soup.find_all('div', class_='tab-content entry-content')[1]
            attribute_list = []
            for a  in attribute_class.strings:
                if a != ' ':
                    attribute_list.append(a.strip())
            key_att = []
            value_att = []
            for index in range(len(attribute_list)):
                if index % 2 != 0:
                    value_att.append(attribute_list[index])
                else: 
                    key_att.append(attribute_list[index])
            for index in range(len(key_att)):
                attribute[key_att[index]] = value_att[index]
            #Get extra_attribute
            extra_class = attribute_soup.find_all('div', class_='tab-content entry-content')[2]
            extra_list = []
            for i in extra_class.strings:
                if i != ' ':
                    extra_list.append(i.strip())
            attribute['Khác'] = '. '.join(extra_list)
            #Get price_house
            price_list = []
            price_tag = attribute_soup.find_all('div',class_='p-price-n')[0].strings
            for i in price_tag:
                if i != ' ':
                    price_list.append(i.strip())
            attribute['Giá'] = price_list[0]
            attribute['Giá/m2'] = price_list[1]
            #Get full attribute and price of district
            full_attribute_list.append(attribute)
            cumsum += 1
            print('Got id',id,'home at',district,'. Total:',cumsum)
        except:
            pass
    
    if len(full_attribute_list) > 0:
        pd.DataFrame(full_attribute_list).to_csv(district+'_p'+str(scale+1)+'.csv',index=False)
    
    return full_attribute_list
    


In [5]:
def get_full_district(full_district,scale=0):
    final = []
    for district in list(full_district.keys())[:]:
        if full_district[district] != 0:
            final += get_attribute_and_price_house(district,full_district[district],scale)
            print('Done:',district)
            sleep(randint(2,10))
    return final

In [6]:
# final_df = get_full_district(total_page,scale=50)